In [1]:
!pip install crewai crewai_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.4/215.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.0/463.0 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install firecrawl-py

People can ask any shopping related questions to this Autonmous AI agent
Note: This Agent works well on US stores only as of now

In [32]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool, ScrapeWebsiteTool

class ShoppingCrew:
    def __init__(self):
        # Initialize tools
        self.search_tool = SerperDevTool()
        self.website_search_tool = WebsiteSearchTool()
        self.scrape_tool = ScrapeWebsiteTool()

    def create_agents(self):
        # Product Verification Agent (New!)
        verification_agent = Agent(
            role='Product Specification Verifier',
            goal='Verify exact product details and model information',
            backstory="""You are a technical specialist who verifies product specifications,
            model numbers, and ensures accuracy of product information. You have extensive
            experience in consumer electronics and can spot incorrect or outdated information.""",
            tools=[self.search_tool, self.website_search_tool],
            verbose=True
        )

        # Research Agent (Enhanced)
        research_agent = Agent(
            role='Product Research Specialist',
            goal='Conduct comprehensive product research including customer reviews and availability',
            backstory="""You are an expert product researcher who specializes in
            detailed product analysis. You focus on gathering authentic customer reviews,
            checking real-time stock availability, and verifying warranty information.
            You know how to distinguish genuine reviews from fake ones and can provide
            balanced perspectives from actual users.""",
            tools=[self.search_tool, self.website_search_tool, self.scrape_tool],
            verbose=True
        )

        # Deal Finding Agent (Enhanced)
        deal_finder_agent = Agent(
            role='Deals Analysis Expert',
            goal='Find and verify current deals with specific dates and conditions',
            backstory="""You are a professional deal finder who specializes in
            tracking promotion dates and conditions. You verify the legitimacy of deals,
            understand complex terms and conditions, and can find hidden savings
            opportunities. You're expert at comparing warranty options and total ownership costs.""",
            tools=[self.search_tool, self.website_search_tool, self.scrape_tool],
            verbose=True
        )

        # Report Generation Agent (Enhanced)
        report_agent = Agent(
            role='Shopping Report Specialist',
            goal='Create detailed, accurate shopping reports with specific links and availability info',
            backstory="""You are an experienced shopping analyst who creates
            comprehensive reports. You ensure all information is current and accurate,
            include specific product links, and provide detailed availability information.
            You excel at presenting complex information in an easy-to-understand format
            while maintaining technical accuracy.""",
            verbose=True
        )

        return verification_agent, research_agent, deal_finder_agent, report_agent

    def create_tasks(self, query: str, verification_agent: Agent, research_agent: Agent,
                    deal_finder_agent: Agent, report_agent: Agent):
        # Verification Task (New!)
        verification_task = Task(
            description=f"""Verify exact product specifications for: {query}

            Required Steps:
            1. Confirm exact model number and name
            2. Verify all technical specifications
            3. Check for any recent product updates or revisions
            4. Confirm product variants and options
            5. Validate compatibility information

            Return:
            - Exact model numbers and names
            - Confirmed specifications
            - Any discrepancies found
            - Latest product updates""",
            agent=verification_agent,
            expected_output="Detailed product verification findings"
        )

        # Research Task (Enhanced)
        research_task = Task(
            description=f"""Research this product in detail: {query}

            Required Steps:
            1. Search for the product across major retailers
            2. Collect and analyze customer reviews (minimum 10)
            3. Check current stock availability
            4. Verify warranty options and terms
            5. Analyze return policies
            6. Compare technical specifications

            Return findings including:
            - Verified retailer list with exact product page URLs
            - Detailed warranty information
            - Stock availability status
            - Curated customer reviews (positive and negative)
            - Technical specifications comparison
            - Return policy details""",
            agent=research_agent,
            context=[verification_task],
            expected_output="""Detailed product research findings which should
             Verified retailer list with exact product page URLs
             Detailed warranty information
             Stock availability status
             Curated customer reviews (positive and negative)
             Technical specifications comparison
             Return policy details
            """
        )

        # Deal Finding Task (Enhanced)
        deal_finding_task = Task(
            description="""Using the research findings, analyze current deals:

            Required Steps:
            1. Verify current prices and promotions
            2. Document specific promotion dates
            3. Detail all terms and conditions
            4. Compare warranty options
            5. Check stock availability for deals
            6. Verify shipping timeframes

            Provide for each retailer:
            - Current price with promotion dates
            - Detailed terms and conditions
            - Warranty options and costs
            - Stock availability status
            - Shipping/pickup options and timeframes
            - Total cost breakdown including all fees""",
            agent=deal_finder_agent,
            context=[verification_task, research_task],
            expected_output="""Detailed deals analysis which should include:
            Current price with promotion dates
            Detailed terms and conditions
            Warranty options and costs
            Stock availability status
            Shipping/pickup options and timeframes
            Total cost breakdown including all fees
            """

        )

        # Report Generation Task (Enhanced)
        report_task = Task(
            description="""Create a comprehensive shopping report including:

            1. Executive Summary
               - Top recommendation with reasoning
               - Price range overview with specific dates
               - Best current deal with terms

            2. Product Details
               - Verified model numbers and specifications
               - Detailed pros and cons
               - Curated customer reviews (minimum 5 positive and 5 negative)
               - Warranty options comparison

            3. Price and Availability Comparison
               - Retailer comparison table with exact URLs
               - Current stock availability
               - Promotion dates and terms
               - Complete cost breakdown

            4. Buying Guide
               - Best retailer choice with reasoning
               - Stock availability alerts
               - Direct product page links
               - Warranty recommendations

            Format the report clearly with sections and bullet points.
            Include specific dates, model numbers, and direct links.""",
            agent=report_agent,
            context=[verification_task, research_task, deal_finding_task],
            expected_output="""Final shopping report which includes clearly with sections and bullet points.
            Include specific dates, model numbers, and direct links"""
        )

        return [verification_task, research_task, deal_finding_task, report_task]

    def run(self, query: str):
        # Create agents
        verification_agent, research_agent, deal_finder_agent, report_agent = self.create_agents()

        # Create tasks
        tasks = self.create_tasks(query, verification_agent, research_agent,
                                deal_finder_agent, report_agent)

        # Create crew
        crew = Crew(
            agents=[verification_agent, research_agent, deal_finder_agent, report_agent],
            tasks=tasks,
            verbose=True,
            process=Process.sequential
        )

        # Start the crew
        result = crew.kickoff()

        return result

# Example usage
if __name__ == "__main__":
    # Set your API keys
    os.environ["OPENAI_API_KEY"] = ""
    os.environ["SERPER_API_KEY"] = ""

    # Create and run the shopping crew
    shopping_crew = ShoppingCrew()
    result = shopping_crew.run("Google Pixel 9 Pro")
    print(result)

# Agent: Product Specification Verifier
## Task: Verify exact product specifications for: Google Pixel 9 Pro
            
            Required Steps:
            1. Confirm exact model number and name
            2. Verify all technical specifications
            3. Check for any recent product updates or revisions
            4. Confirm product variants and options
            5. Validate compatibility information
            
            Return:
            - Exact model numbers and names
            - Confirmed specifications
            - Any discrepancies found
            - Latest product updates


# Agent: Product Specification Verifier
## Thought: I need to find detailed information regarding the Google Pixel 9 Pro, including its model number, technical specifications, any recent updates, product variants, and compatibility information.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Google Pixel 9 Pro specifications and model number\"}"
## Tool Output:

Inserting batches in chromadb: 100%|██████████| 2/2 [00:01<00:00,  1.06it/s]




# Agent: Product Specification Verifier
## Thought: Thought: I will now extract the most relevant information regarding the recent updates, revisions, product variants, and compatibility information for the Google Pixel 9 Pro to conclude my verification.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"Google Pixel 9 Pro product updates\", \"website\": \"https://support.google.com/pixelphone/answer/7158570?hl=en\"}"
## Tool Output: 
Relevant Content:
Tech Specs of Pixel 9 Pro & Pixel 9 Pro XLIt’s that time of year. Shop Black Friday dealsPhonesEarbudsTabletsWatches & TrackersSmart HomeAccessoriesOffersSupportOrdersPhonesEarbudsTabletsWatches & TrackersSmart HomeAccessoriesOffersSupport0Sign in with GoogleOrdersRepairsPreferred CareSubscriptionsMy Nest HomeSettingsPixel 9 ProFrom $849 $999or $23.58/month with 36-month financing*BuyFrom $849 $999or $23.58/month with 36-month financing*BuyExplore the specs.Pixel 9 ProPixel 9 Pro XLColorsDisplayDimensions

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.53it/s]




# Agent: Product Research Specialist
## Thought: Thought: I need to gather customer reviews from the retailer pages I just found. I'll start by checking customer reviews from Amazon and Best Buy for the Google Pixel 9 Pro and understand stock availability for these retailers.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"Google Pixel 9 Pro customer reviews\", \"website\": \"https://www.amazon.com/Google-Pixel-Pro-Unlocked-Smartphone/dp/B0D7HVCD91\"}"
## Tool Output: 
Relevant Content:
videoUnboxing Google Pixel 9 Pro ASMR.! Collins Reviews Videos for this product 0:12 Click to play videoPixel 9 Pro XL with Clear Case Tight Fit ErsRev Videos for this product 8:55 Click to play videoGoogle Pixel 9 Pro XL Unboxing and Test Influencer Video Videos for this product 0:19 Click to play videoZoom on the Google Pixel 9 Pro Jessie Renner Videos for this product 1:20 Click to play videoWe unboxed - Here are the results! 🟢 Product Breakdown Next pageUpload you

...



# Agent: Product Research Specialist
## Thought: Thought: Now that I have information on the product reviews and warranty details, I will retrieve the return policy information next.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"Google Pixel 9 Pro return policy\", \"website\": \"https://www.amazon.com/Google-Pixel-Pro-Unlocked-Smartphone/dp/B0D7HVCD91\"}"
## Tool Output: 
Relevant Content:
Skip to main content Amazon.com: Google Pixel 9 Pro - Unlocked Android Smartphone with Gemini, Triple Rear Camera System, 24-Hour Battery, and 6.3" Super Actua Display - Obsidian - 128 GB : Cell Phones & Accessories Cell Phones & Accessories › Cell Phones Add to your order Coverage for accidental damage including drops, spills, and broken parts, as well as breakdowns (plans vary) 24/7 support when you need it. Quick, easy, and frustration-free claims. Cover this product: 2-Year Protection Plan $199.99 Learn more 2 Year Mobile Phone Accident Protection Plan from A

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  2.81it/s]




# Agent: Deals Analysis Expert
## Thought: I need to gather detailed deals and terms for the Google Pixel 9 Pro from the verified retailers, specifically Amazon and Best Buy. I will first search for current prices, promotions, and all relevant information for these retailers.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"Google Pixel 9 Pro current price and promotions\", \"website\": \"https://www.amazon.com/\"}"
## Tool Output: 
Relevant Content:
Tech Specs of Pixel 9 Pro & Pixel 9 Pro XLIt’s that time of year. Shop Black Friday dealsPhonesEarbudsTabletsWatches & TrackersSmart HomeAccessoriesOffersSupportOrdersPhonesEarbudsTabletsWatches & TrackersSmart HomeAccessoriesOffersSupport0Sign in with GoogleOrdersRepairsPreferred CareSubscriptionsMy Nest HomeSettingsPixel 9 ProFrom $849 $999or $23.58/month with 36-month financing*BuyFrom $849 $999or $23.58/month with 36-month financing*BuyExplore the specs.Pixel 9 ProPixel 9 Pro XLColorsDisplayDimensions

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  4.14it/s]




# Agent: Deals Analysis Expert
## Thought: I will now focus on gathering detailed information from Best Buy regarding the Google Pixel 9 Pro, including prices, promotions, terms, warranty options, stock status, and shipping terms.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"Google Pixel 9 Pro current price and promotions\", \"website\": \"https://www.bestbuy.com/\"}"
## Tool Output: 
Relevant Content:
Tech Specs of Pixel 9 Pro & Pixel 9 Pro XLIt’s that time of year. Shop Black Friday dealsPhonesEarbudsTabletsWatches & TrackersSmart HomeAccessoriesOffersSupportOrdersPhonesEarbudsTabletsWatches & TrackersSmart HomeAccessoriesOffersSupport0Sign in with GoogleOrdersRepairsPreferred CareSubscriptionsMy Nest HomeSettingsPixel 9 ProFrom $849 $999or $23.58/month with 36-month financing*BuyFrom $849 $999or $23.58/month with 36-month financing*BuyExplore the specs.Pixel 9 ProPixel 9 Pro XLColorsDisplayDimensions and WeightBattery and ChargingMemory and Sto